In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

print("Reading parquet file...")
trips_df = pd.read_parquet('green_tripdata_2025-11.parquet')
print(f"Trips data shape: {trips_df.shape}")

print("\nReading zones CSV...")
zones_df = pd.read_csv('taxi_zone_lookup.csv')
print(f"Zones data shape: {zones_df.shape}")

Reading parquet file...
Trips data shape: (46912, 21)

Reading zones CSV...
Zones data shape: (265, 4)


In [4]:
zones_df.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [5]:
trips_df.head() 

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1.0,74,42,1.0,0.74,7.2,...,0.5,1.94,0.0,NaN,1.0,11.64,1.0,1.0,0.00,0.0
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1.0,74,42,2.0,0.95,7.2,...,0.5,0.00,0.0,NaN,1.0,9.70,2.0,1.0,0.00,0.0
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1.0,83,160,1.0,2.19,13.5,...,0.5,5.00,0.0,NaN,1.0,21.00,1.0,1.0,0.00,0.0
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1.0,166,127,1.0,5.44,24.7,...,0.5,0.50,0.0,NaN,1.0,27.70,1.0,1.0,0.00,0.0
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1.0,166,262,1.0,3.20,18.4,...,1.5,1.00,0.0,NaN,1.0,24.65,1.0,1.0,2.75,0.0


In [6]:
#For the trips in November 2025 (lpep_pickup_datetime between 
#'2025-11-01' and '2025-12-01', exclusive of the upper bound), how many trips had a trip_distance of less than or equal to 1 mile?
nov_trips = trips_df[
    (trips_df['lpep_pickup_datetime'] >= '2025-11-01') &
    (trips_df['lpep_pickup_datetime'] < '2025-12-01') &
    (trips_df['trip_distance'] <= 1)
]
print(f"\nNumber of trips in Nov 2025 with distance <= 1 mile: {len(nov_trips)}")


Number of trips in Nov 2025 with distance <= 1 mile: 8007


In [7]:
pickup_day = trips_df[trips_df['trip_distance'] < 100]['lpep_pickup_datetime'].dt.date.iloc[trips_df['trip_distance'].idxmax()]
pickup_day

datetime.date(2025, 11, 1)

In [8]:
# Filter trips with distance < 100
filtered_trips = trips_df[trips_df['trip_distance'] < 100]

# Find the row with maximum distance
max_distance_row = filtered_trips.loc[filtered_trips['trip_distance'].idxmax()]

# Get the pickup day
pickup_day = max_distance_row['lpep_pickup_datetime'].date()
print(f"pick up day with the longest trip distance : {pickup_day}")

pick up day with the longest trip distance : 2025-11-14


In [17]:
total_amount = trips_df[(trips_df['lpep_pickup_datetime'].dt.date == pd.Timestamp('2025-11-18').date())
].groupby('PULocationID')['total_amount'].sum()

# Get the PULocationID with maximum total_amount (1st place)
zone_name = zones_df['Zone'][zones_df['LocationID'] == total_amount.idxmax()].iloc[0]
print(f"Top pickup zone: {zone_name}")

Top pickup zone: East Harlem North


In [18]:
#Question 6. For the passengers picked up in the zone named "East Harlem North" in 
#November 2025, which was the drop off zone that had the largest tip? (1 point)
east_harlem_id = zones_df.loc[zones_df['Zone'] == 'East Harlem North', 'LocationID'].values[0]
result = (
    trips_df
    .query("lpep_pickup_datetime >= '2025-11-01' and lpep_pickup_datetime < '2025-12-01'")
    .query(f"PULocationID == {east_harlem_id}")
    .groupby('DOLocationID')['tip_amount']
    .max()
    .idxmax()
)

dropoff_zone = zones_df.loc[zones_df['LocationID'] == result, 'Zone'].values[0]
print(f"Drop-off zone with largest tip: {dropoff_zone}")

Drop-off zone with largest tip: Yorkville West
